In [ ]:
from grid.eval import *

In [ ]:
boardsize = 3
n_workers = 1

snaps = snapshots(boardsize)
snaps = pd.concat([snaps, parameters(snaps)], 1)
snaps['nickname'] = snaps.run.str.extract('.* (.*)', expand=False) + '.' + snaps.idx.astype(str)
snaps['params'] = params(snaps)
snaps = snaps.set_index('nickname')

n_agents = len(snaps)
wins  = pd.DataFrame(np.zeros((n_agents, n_agents)), snaps.index, snaps.index)
games = pd.DataFrame(np.zeros((n_agents, n_agents)), snaps.index, snaps.index)

soln = None
futures = {}
with ProcessPoolExecutor(n_workers) as pool:
    while True:
        try:
            soln = activelo.solve(games, wins, soln=soln)
        except InManifoldError:
            soln = None
            log.warning('Got a manifold error; throwing soln out')

        for key, future in list(futures.items()):
            if future.done():
                results = future.result()
                games, wins = update(games, wins, results)
                del futures[key]

        while len(futures) < n_workers:
            if soln is None:
                sugg = tuple(np.random.choice(games.index, (2,)))
            else:
                sugg = suggest(soln, n_workers)
            
            futures[sugg] = pool.submit(evaluate, *sugg)
    
        if soln is not None:
            report(soln, games, futures)
            _, σ = arena.analysis.difference(soln, soln.μ.idxmin())
            if σ.pow(2).mean()**.5 < .1:
                break
        
snaps['μ'], snaps['σ'] = arena.analysis.difference(soln, soln.μ.idxmin())



In [ ]:
while True:
    try:
        soln = activelo.solve(games, wins, soln=soln)
    except InManifoldError:
        soln = activelo.solve(games, wins)
    sugg = activelo.suggest(soln)

    results = evaluate(*sugg)
    games, wins = update(games, wins, results)

    if soln is not None:
        report(soln, games, futures)
        μ, σ = arena.analysis.difference(soln, soln.μ.idxmin())
        if σ.pow(2).mean()**.5 < .01:
            break
        
snaps['μ'], snaps['σ'] = arena.analysis.difference(soln, soln.μ.idxmin())

In [ ]:
soln = activelo.solve(games, wins, soln=soln)
snaps['μ'], snaps['σ'] = arena.analysis.difference(soln, soln.μ.idxmin())

In [ ]:
snaps.μ.plot()

In [ ]:
(wins/games)

In [ ]:
evaluate('arctic-ease.0', 'arctic-ease.20')

In [ ]:
results

In [ ]:
snaps